In [0]:
%run "../includes/configuration"

In [0]:
from pyspark.sql.functions import current_timestamp
def add_ingestion_date(input_df):
  output_df = input_df.withColumn('ingestion_date',current_timestamp())
  return output_df

In [0]:
def df_column_to_list(input_df, column_name):
  df_row_list = input_df.select(column_name) \
                        .distinct() \
                        .collect()
  
  column_value_list = [row[column_name] for row in df_row_list]
  return column_value_list

In [0]:

def merge_delta_data(input_df,db_name,table_name,merge_condition,partition_column):
    spark.conf.set("spark.databricks.optimizer.dynamicPartitionPruning", "true")
    from delta.tables import DeltaTable
    if(spark.catalog.tableExists(f"{db_name}.{table_name}")):
        deltaTable = DeltaTable.forName(spark, f"{db_name}.{table_name}")
        deltaTable.alias("tgt").merge(
        input_df.alias("src"),
        merge_condition) \
        .whenMatchedUpdateAll() \
        .whenNotMatchedInsertAll()\
            .execute()
    else:
        input_df.write.mode("overwrite").partitionBy(partition_column).format("delta").saveAsTable(f"{db_name}.{table_name}")


In [0]:
entity_config = {
    "circuits":     {"endpoint": "circuits", "scope": "global"},
    "constructors": {"endpoint": "constructors", "scope": "global"},
    "drivers":      {"endpoint": "drivers", "scope": "global"},
    "races":        {"endpoint": "races", "scope": "season"},
    "results":      {"endpoint": "results", "scope": "season"},
    "qualifying":   {"endpoint": "qualifying", "scope": "season"},
    "pitstops":     {"endpoint": "pitstops", "scope": "round"},
    "laps":         {"endpoint": "laps", "scope": "round"}
}

dbutils.widgets.text("p_entity", "circuits")
dbutils.widgets.text("p_year", "2025")
dbutils.widgets.text("p_round", "1")

v_entity = dbutils.widgets.get("p_entity")
v_year = dbutils.widgets.get("p_year")
v_round = dbutils.widgets.get("p_round")